In [6]:
import concurrent.futures
import importlib
import openai
import os
import requests
import bs4
from bs4 import BeautifulSoup
from itertools import islice
openai.organization = "org-RfxFQjm7zizJjdQRALbJZaZB"
openai.api_key = os.environ.get("OPENAI_API_KEY")
modelToUse = "gpt-3.5-turbo"
print(modelToUse)

import scripts.marbot as marbot

gpt-3.5-turbo


In [31]:
importlib.reload(marbot) # little hack since I jump back and forth
marbot.loadUrls()

Fetching https://www.oemoffhighway.com/trends/document/22578646/2022-state-of-the-industry-oem-offhighway
Saved 121715 chars to /app/testdata/Equipment-0.html
Fetching https://www.oemoffhighway.com/trends/article/22552113/oem-offhighway-state-of-the-industry-2022
Saved 154814 chars to /app/testdata/Equipment-1.html
Fetching https://www.deskera.com/blog/plastic-manufacturing-key-trends-and-innovations/amp/
Saved 56738 chars to /app/testdata/Plastic-0.html
Fetching https://www.deskera.com/blog/plastic-manufacturing-critical-issues-and-challenges/amp/
Saved 81687 chars to /app/testdata/Plastic-1.html
Fetching https://www.fortunebusinessinsights.com/amp/plastics-market-102176
Saved 85178 chars to /app/testdata/Plastic-2.html
Fetching https://www.kaysun.com/blog/plastics-industry-trends
Saved 128125 chars to /app/testdata/Plastic-3.html
Fetching https://www.manufacturing.net/operations/article/13057614/the-future-of-plastics-manufacturing-in-the-us
Saved 237551 chars to /app/testdata/Plasti

In [39]:
importlib.reload(marbot) # little hack since I jump back and forth
filePaths = marbot.scanFiles("/app/testdata")
filePaths


{'Aircraft': ['/app/testdata/Aircraft-3.html',
  '/app/testdata/Aircraft-11.html',
  '/app/testdata/Aircraft-10.html',
  '/app/testdata/Aircraft-2.html',
  '/app/testdata/Aircraft-9.html',
  '/app/testdata/Aircraft-5.html',
  '/app/testdata/Aircraft-4.html',
  '/app/testdata/Aircraft-8.html',
  '/app/testdata/Aircraft-7.html',
  '/app/testdata/Aircraft-6.html',
  '/app/testdata/Aircraft-1.html',
  '/app/testdata/Aircraft-0.html'],
 'Paper': ['/app/testdata/Paper-6.html',
  '/app/testdata/Paper-7.html',
  '/app/testdata/Paper-0.html',
  '/app/testdata/Paper-2.html',
  '/app/testdata/Paper-3.html',
  '/app/testdata/Paper-4.html',
  '/app/testdata/Paper-8.html',
  '/app/testdata/Paper-9.html',
  '/app/testdata/Paper-5.html'],
 'Bakerybread': ['/app/testdata/Bakerybread-10.html',
  '/app/testdata/Bakerybread-0.html',
  '/app/testdata/Bakerybread-1.html',
  '/app/testdata/Bakerybread-6.html',
  '/app/testdata/Bakerybread-7.html',
  '/app/testdata/Bakerybread-4.html',
  '/app/testdata/Bakery

In [14]:
print(f"{len(pageSummaries)}: {list(len(x) for x in pageSummaries)}")

4: [1363, 2173, 328, 1662]


In [25]:
importlib.reload(marbot) # little hack since I jump back and forth
combinedSummaries = marbot.combinePageSummaries(pageSummaries, "gpt-4")



Combined page summaries:
"Interconnected factories" (3): The plastics industry is seeing increased use of interconnected or "smart" factories for better efficiency and quality control. This is part of a broader trend toward automation and customization technologies in manufacturing.

"Increasing Demand for Engineering Plastics" (3): There is an observed increase in demand for engineering plastics, polymer solutions, and metal substitution. 

"Environmental sustainability" (3): With global awareness increasing towards the harmful effects of plastic pollution, there's a significant focus on reducing environmental impact. Practices range from light-weighting, the use of sustainable materials, to recycling programs and an industry-wide shift towards renewable sources and recyclable products.  

"Challenges by regulations" (2): There's an increasing societal and governmental pressure on the industry to reduce plastic pollution. This results in more stringent regulations and standards whic